# Importating  Packages

In [2]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import re
import warnings
warnings.filterwarnings('ignore')
# set plot style
sns.set()


from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
#from gensim.models.fasttext import FastText
import string
from nltk import SnowballStemmer

import contractions
#import Fasttext
from nltk.tokenize import word_tokenize,TreebankWordTokenizer
from nltk.corpus import stopwords, wordnet
plt.xticks(rotation=70)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline

import nltk
import pickle
import pyLDAvis.sklearn

import pyLDAvis.sklearn
from collections import Counter
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)

# Data Loading 

In [3]:
df_train= pd.read_csv("train.csv")
df_test= pd.read_csv("test_with_no_labels.csv")

In [4]:
df_train.head()

,sentiment,message,tweetid
0,1,"PolySciMajor EPA chief doesn't think carbon dioxide is main cause of global warming and.. wait, ...",625221
1,1,It's not like we lack evidence of anthropogenic global warming,126103
2,2,RT @RawStory: Researchers say we have three years to act on climate change before it’s too late ...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year in the war on climate change https://t.co/44wOTxTLcD,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, sexist, climate change denying bigot is leading in...",466954


## Text Pre processing

In [5]:
#Check the number of column and rows available in our dataset
print('Shape of train data - ', df_train.shape)
print('Shape of test data - ', df_test.shape)

Shape of train data -  (15819, 3)
Shape of test data -  (10546, 2)


In [6]:
# Checking through the list item available in our columns datasets
print('df_train columns:')
df_train.columns

df_train columns:


Index(['sentiment', 'message', 'tweetid'], dtype='object')

**Note :Checking for the null value in the train Dataset**

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15819 entries, 0 to 15818
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  15819 non-null  int64 
 1   message    15819 non-null  object
 2   tweetid    15819 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 370.9+ KB


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10546 entries, 0 to 10545
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   message  10546 non-null  object
 1   tweetid  10546 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 164.9+ KB


## Observation

1.It was observed that the df_train dataset contains 3 columns and 15819 rows.

2.There were no null value present in the dataset

3.The dataset has both numerical and categorical datatype.

4.Both Df_train and Df_test dataset for message needs to be worked upon because such categorical is not acceptable in building a model.

**10 Sample Observation**

In [9]:
df_train[[x for x in df_train.columns if df_train[x].dtype == 'object']].sample(10)

,message
2006,"RT @kwilli1046: Attention all liberals, Macron offers you “refugee” status. Please move and leav..."
13651,RT @SenSanders: We have a president-elect who doesn't believe in climate change. Millions of peo...
8512,RT @StephenSchlegel: she's thinking about how she's going to die because your husband doesn't be...
11595,"RT @dwallacewells: How bad could climate change get? Really, really bad, and really really fast...."
5201,@KHOUBlake11 @jcdrex @FoxNews @CNN @weatherchannel @GaughanSurfing 500 Year floods. No climate c...
14137,RT @TheCartelMatt: https://t.co/F3IZMl6dEM The left is upset that Trump is taking out terrorists...
9343,"RT @ddlovato: #IVotedBecause I believe in equality, we need comprehensive immigration reform, I ..."
6349,RT @nobby15: Twitter resistance explodes after federal climate change gag order https://t.co/svq...
12529,1 News Ã¢â‚¬Â¢Ã‚ 'Ã¢â‚¬ËœFight against climate change is a moral obligationÃ¢â‚¬â„¢' via @233liv...
4044,How scientists plan to avoid another global warming Ã¢â‚¬Å“pauseÃ¢â‚¬ï† row: Scientists need to...


In [10]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
sentiment,15819.0,0.917504,0.836537,-1.0,1.0,1.0,1.0,2.0
tweetid,15819.0,501719.433656,289045.983132,6.0,253207.5,502291.0,753769.0,999888.0


In [11]:
df_train.skew()

sentiment   -0.683959
tweetid     -0.008471
dtype: float64

In [12]:
df_train.kurtosis()

sentiment    0.122976
tweetid     -1.193356
dtype: float64

In [13]:
def delete_url(data, col):
    """
        Accepts a dataframe and col., removes web urls from the col.
        returns a new dataframe 
    """
    df = data.copy()
    pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
    subs_url = ''
    df[col] = df[col].replace(to_replace = pattern_url, value = subs_url, regex = True)
    return df

In [14]:
# Create a new dataframe with message colun void of url links
new_df_train = delete_url(df_train, 'message')
new_df_train.head()

,sentiment,message,tweetid
0,1,"PolySciMajor EPA chief doesn't think carbon dioxide is main cause of global warming and.. wait, ...",625221
1,1,It's not like we lack evidence of anthropogenic global warming,126103
2,2,RT @RawStory: Researchers say we have three years to act on climate change before it’s too late …,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year in the war on climate change,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, sexist, climate change denying bigot is leading in...",466954


In [15]:
def delete_tags(data, col):
    """
        This function takes in a dataframe and a col, removes all words started with '#' and '@' in the column,
        and returns a new dataframe
    """
    df = data.copy()
    pattern_tags = r'#\w+[#?]'
    pattern_2 = r'@\w+'
    subs_tag = ''
    df[col] = df[col].replace(to_replace = pattern_tags, value = subs_tag, regex = True)
    df[col] = df[col].replace(to_replace = pattern_2, value = subs_tag, regex = True)
    return df

In [16]:
# Create a new dataframe with message colun void of url links
new_df_train = delete_tags(new_df_train, 'message')
new_df_train.head()

,sentiment,message,tweetid
0,1,"PolySciMajor EPA chief doesn't think carbon dioxide is main cause of global warming and.. wait, ...",625221
1,1,It's not like we lack evidence of anthropogenic global warming,126103
2,2,RT : Researchers say we have three years to act on climate change before it’s too late …,698562
3,1,WIRED : 2016 was a pivotal year in the war on climate change,573736
4,1,"RT : It's 2016, and a racist, sexist, climate change denying bigot is leading in the polls. #Ele...",466954


In [17]:
def remove_punc(data, col):
    """
        This function takes in a dataframe and a column, uses python string package to identify and remove all
        punctions in the column. It returns a new dataframe
    """
    def operation(post):
        return ''.join([l for l in post if l not in string.punctuation])
    
    df = data.copy()
    
    df[col] = df[col].apply(operation)
    return df

In [18]:
# Create a new dataframe with the message colmn void of punctuations
new_df_train = remove_punc(new_df_train, 'message')
new_df_train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesnt think carbon dioxide is main cause of global warming and wait what...,625221
1,1,Its not like we lack evidence of anthropogenic global warming,126103
2,2,RT Researchers say we have three years to act on climate change before it’s too late …,698562
3,1,WIRED 2016 was a pivotal year in the war on climate change,573736
4,1,RT Its 2016 and a racist sexist climate change denying bigot is leading in the polls ElectionNight,466954


In [19]:
# Create a new dataframe with message colun void of url links
new_df_train = delete_url(new_df_train, 'message')
new_df_train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesnt think carbon dioxide is main cause of global warming and wait what...,625221
1,1,Its not like we lack evidence of anthropogenic global warming,126103
2,2,RT Researchers say we have three years to act on climate change before it’s too late …,698562
3,1,WIRED 2016 was a pivotal year in the war on climate change,573736
4,1,RT Its 2016 and a racist sexist climate change denying bigot is leading in the polls ElectionNight,466954


In [20]:
def word_converter(data, col):
    """
        This function takes in a dataframe and col, converts all capitalized words in the column to lowercase,
        and returns a new dataframe.
    """
    df = data.copy()
    df[col] = df[col].str.lower()
    return df

In [21]:
new_df_train = word_converter(new_df_train, 'message')
new_df_train.head()

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesnt think carbon dioxide is main cause of global warming and wait what...,625221
1,1,its not like we lack evidence of anthropogenic global warming,126103
2,2,rt researchers say we have three years to act on climate change before it’s too late …,698562
3,1,wired 2016 was a pivotal year in the war on climate change,573736
4,1,rt its 2016 and a racist sexist climate change denying bigot is leading in the polls electionnight,466954


In [23]:
def remove_new_line(data, col):
    """
        Takes in a dataframe and a column, returns a new dataframe with a new column void of new line command
    """

    def operation(text):
        result = re.sub("\n", "", text)
        result = re.sub("rt", "", result)
        return result

    df = data.copy()
    
    df[col] = df[col].apply(operation)

    return df

In [24]:
# Create a new dataframe with the message colmn void of punctuations
new_df_train = remove_new_line(new_df_train, 'message')
new_df_train.head()

,sentiment,message,tweetid
0,1,polyscimajor epa chief doesnt think carbon dioxide is main cause of global warming and wait what...,625221
1,1,its not like we lack evidence of anthropogenic global warming,126103
2,2,researchers say we have three years to act on climate change before it’s too late …,698562
3,1,wired 2016 was a pivotal year in the war on climate change,573736
4,1,its 2016 and a racist sexist climate change denying bigot is leading in the polls electionnight,466954


new_df_train.drop('tokenized', axis=1, inplace=True)

In [25]:
def tokenizer(data, col):
    """
        This function takes in a dataframe and a col, creates a new column to store the tokenized words
        in the inputed column, and returns a new dataframe.
    """
    df = data.copy()
    tokeniser = TreebankWordTokenizer()
    df['Tokenized'] = df[col].apply(tokeniser.tokenize)
    return df

In [26]:
# Create a new column to hold the tokens from message column
new_df_train = tokenizer(new_df_train, 'message')
new_df_train.head()

,sentiment,message,tweetid,Tokenized
0,1,polyscimajor epa chief doesnt think carbon dioxide is main cause of global warming and wait what...,625221,"[polyscimajor, epa, chief, doesnt, think, carbon, dioxide, is, main, cause, of, global, warming,..."
1,1,its not like we lack evidence of anthropogenic global warming,126103,"[its, not, like, we, lack, evidence, of, anthropogenic, global, warming]"
2,2,researchers say we have three years to act on climate change before it’s too late …,698562,"[researchers, say, we, have, three, years, to, act, on, climate, change, before, it’s, too, late..."
3,1,wired 2016 was a pivotal year in the war on climate change,573736,"[wired, 2016, was, a, pivotal, year, in, the, war, on, climate, change]"
4,1,its 2016 and a racist sexist climate change denying bigot is leading in the polls electionnight,466954,"[its, 2016, and, a, racist, sexist, climate, change, denying, bigot, is, leading, in, the, polls..."


In [27]:
df = new_df_train.copy()

In [32]:
def stem_words(data, col):
    """
        Takes in a dataframe and a column, converts the words in the column to it root form,
        with the aid of SnowballStemmer class from the nltk package.
        Returns a new dataframe with an additional column "message_stem"
    """
    stemmer = SnowballStemmer('english')
    def operation(words, stemmer):
        return [stemmer.stem(word) for word in words]
    
    df = data.copy()
    df["stemmed word"] = df[col].apply(operation, args=(stemmer, ))
    
    return df

In [33]:
# Create a new column to hold root words from stemmer
new_df_train = stem_words(new_df_train, 'Tokenized')
new_df_train.head()

,sentiment,message,tweetid,Tokenized,stemmed word
0,1,polyscimajor epa chief doesnt think carbon dioxide is main cause of global warming and wait what...,625221,"[polyscimajor, epa, chief, doesnt, think, carbon, dioxide, is, main, cause, of, global, warming,...","[polyscimajor, epa, chief, doesnt, think, carbon, dioxid, is, main, caus, of, global, warm, and,..."
1,1,its not like we lack evidence of anthropogenic global warming,126103,"[its, not, like, we, lack, evidence, of, anthropogenic, global, warming]","[it, not, like, we, lack, evid, of, anthropogen, global, warm]"
2,2,researchers say we have three years to act on climate change before it’s too late …,698562,"[researchers, say, we, have, three, years, to, act, on, climate, change, before, it’s, too, late...","[research, say, we, have, three, year, to, act, on, climat, chang, befor, it, too, late, …]"
3,1,wired 2016 was a pivotal year in the war on climate change,573736,"[wired, 2016, was, a, pivotal, year, in, the, war, on, climate, change]","[wire, 2016, was, a, pivot, year, in, the, war, on, climat, chang]"
4,1,its 2016 and a racist sexist climate change denying bigot is leading in the polls electionnight,466954,"[its, 2016, and, a, racist, sexist, climate, change, denying, bigot, is, leading, in, the, polls...","[it, 2016, and, a, racist, sexist, climat, chang, deni, bigot, is, lead, in, the, poll, election..."


In [34]:
def lam_words(data, col):
    """
        Takes in a dataframe and a column, converts the words in the column to it root form,
        with the aid of WordNetLemmatizer class from the nltk package.
        Returns a new dataframe with an additional column "message_lam"
    """
    lemmatizer = WordNetLemmatizer()
    def operation(words, lemmatizer):
        return [lemmatizer.lemmatize(word) for word in words] 
    df = data.copy()
    df["lemmatized"] = df[col].apply(operation, args=(lemmatizer, ))
    
    return df

new_df_train.drop('message_lam', axis=1, inplace=True)

In [35]:
# Create a new column to hold root words from stemmer
new_df_train = lam_words(new_df_train, 'Tokenized')
new_df_train.head()

,sentiment,message,tweetid,Tokenized,stemmed word,lemmatized
0,1,polyscimajor epa chief doesnt think carbon dioxide is main cause of global warming and wait what...,625221,"[polyscimajor, epa, chief, doesnt, think, carbon, dioxide, is, main, cause, of, global, warming,...","[polyscimajor, epa, chief, doesnt, think, carbon, dioxid, is, main, caus, of, global, warm, and,...","[polyscimajor, epa, chief, doesnt, think, carbon, dioxide, is, main, cause, of, global, warming,..."
1,1,its not like we lack evidence of anthropogenic global warming,126103,"[its, not, like, we, lack, evidence, of, anthropogenic, global, warming]","[it, not, like, we, lack, evid, of, anthropogen, global, warm]","[it, not, like, we, lack, evidence, of, anthropogenic, global, warming]"
2,2,researchers say we have three years to act on climate change before it’s too late …,698562,"[researchers, say, we, have, three, years, to, act, on, climate, change, before, it’s, too, late...","[research, say, we, have, three, year, to, act, on, climat, chang, befor, it, too, late, …]","[researcher, say, we, have, three, year, to, act, on, climate, change, before, it’s, too, late, …]"
3,1,wired 2016 was a pivotal year in the war on climate change,573736,"[wired, 2016, was, a, pivotal, year, in, the, war, on, climate, change]","[wire, 2016, was, a, pivot, year, in, the, war, on, climat, chang]","[wired, 2016, wa, a, pivotal, year, in, the, war, on, climate, change]"
4,1,its 2016 and a racist sexist climate change denying bigot is leading in the polls electionnight,466954,"[its, 2016, and, a, racist, sexist, climate, change, denying, bigot, is, leading, in, the, polls...","[it, 2016, and, a, racist, sexist, climat, chang, deni, bigot, is, lead, in, the, poll, election...","[it, 2016, and, a, racist, sexist, climate, change, denying, bigot, is, leading, in, the, poll, ..."


**Removing stopwords**

In [36]:
def remove_stop_words(data, col):
    """
        Takes a dataframe and a column, creates a new dataframe with a new column no_stop_word from the input
        dataframe and column, returns the new column
    """
    def operation(toks):
        new_toks = [tok for tok in toks if tok not in stopwords.words('english')]
        return new_toks
    
    df = data.copy()
    df['stop word removed'] = df[col].apply(operation)

    return df

In [37]:
# Create a new column from message_lam void of stop words
new_df_train = remove_stop_words(new_df_train, 'Tokenized')
new_df_train.head()

,sentiment,message,tweetid,Tokenized,stemmed word,lemmatized,stop word removed
0,1,polyscimajor epa chief doesnt think carbon dioxide is main cause of global warming and wait what...,625221,"[polyscimajor, epa, chief, doesnt, think, carbon, dioxide, is, main, cause, of, global, warming,...","[polyscimajor, epa, chief, doesnt, think, carbon, dioxid, is, main, caus, of, global, warm, and,...","[polyscimajor, epa, chief, doesnt, think, carbon, dioxide, is, main, cause, of, global, warming,...","[polyscimajor, epa, chief, doesnt, think, carbon, dioxide, main, cause, global, warming, wait, via]"
1,1,its not like we lack evidence of anthropogenic global warming,126103,"[its, not, like, we, lack, evidence, of, anthropogenic, global, warming]","[it, not, like, we, lack, evid, of, anthropogen, global, warm]","[it, not, like, we, lack, evidence, of, anthropogenic, global, warming]","[like, lack, evidence, anthropogenic, global, warming]"
2,2,researchers say we have three years to act on climate change before it’s too late …,698562,"[researchers, say, we, have, three, years, to, act, on, climate, change, before, it’s, too, late...","[research, say, we, have, three, year, to, act, on, climat, chang, befor, it, too, late, …]","[researcher, say, we, have, three, year, to, act, on, climate, change, before, it’s, too, late, …]","[researchers, say, three, years, act, climate, change, it’s, late, …]"
3,1,wired 2016 was a pivotal year in the war on climate change,573736,"[wired, 2016, was, a, pivotal, year, in, the, war, on, climate, change]","[wire, 2016, was, a, pivot, year, in, the, war, on, climat, chang]","[wired, 2016, wa, a, pivotal, year, in, the, war, on, climate, change]","[wired, 2016, pivotal, year, war, climate, change]"
4,1,its 2016 and a racist sexist climate change denying bigot is leading in the polls electionnight,466954,"[its, 2016, and, a, racist, sexist, climate, change, denying, bigot, is, leading, in, the, polls...","[it, 2016, and, a, racist, sexist, climat, chang, deni, bigot, is, lead, in, the, poll, election...","[it, 2016, and, a, racist, sexist, climate, change, denying, bigot, is, leading, in, the, poll, ...","[2016, racist, sexist, climate, change, denying, bigot, leading, polls, electionnight]"


In [38]:
df = new_df_train.copy()

In [39]:
def form_corpus(data, col):
    """
        This function takes in a dataframe and a column with tokenized text, 
        returns a new dataframe with an additional column(delimeter removed), which is made up of all words in the inserted colunm
        but void of delimeters.
    """
    def operation(tok_list):
        string = ' '.join(tok_list)
        return string
    df = data.copy()
    df['delimeter removed'] = df[col].apply(operation)
    
    return df

In [40]:
# Create a new column from no_stop_word void of delimeters
new_df_train = form_corpus(new_df_train, 'stop word removed')
new_df_train.head()

,sentiment,message,tweetid,Tokenized,stemmed word,lemmatized,stop word removed,delimeter removed
0,1,polyscimajor epa chief doesnt think carbon dioxide is main cause of global warming and wait what...,625221,"[polyscimajor, epa, chief, doesnt, think, carbon, dioxide, is, main, cause, of, global, warming,...","[polyscimajor, epa, chief, doesnt, think, carbon, dioxid, is, main, caus, of, global, warm, and,...","[polyscimajor, epa, chief, doesnt, think, carbon, dioxide, is, main, cause, of, global, warming,...","[polyscimajor, epa, chief, doesnt, think, carbon, dioxide, main, cause, global, warming, wait, via]",polyscimajor epa chief doesnt think carbon dioxide main cause global warming wait via
1,1,its not like we lack evidence of anthropogenic global warming,126103,"[its, not, like, we, lack, evidence, of, anthropogenic, global, warming]","[it, not, like, we, lack, evid, of, anthropogen, global, warm]","[it, not, like, we, lack, evidence, of, anthropogenic, global, warming]","[like, lack, evidence, anthropogenic, global, warming]",like lack evidence anthropogenic global warming
2,2,researchers say we have three years to act on climate change before it’s too late …,698562,"[researchers, say, we, have, three, years, to, act, on, climate, change, before, it’s, too, late...","[research, say, we, have, three, year, to, act, on, climat, chang, befor, it, too, late, …]","[researcher, say, we, have, three, year, to, act, on, climate, change, before, it’s, too, late, …]","[researchers, say, three, years, act, climate, change, it’s, late, …]",researchers say three years act climate change it’s late …
3,1,wired 2016 was a pivotal year in the war on climate change,573736,"[wired, 2016, was, a, pivotal, year, in, the, war, on, climate, change]","[wire, 2016, was, a, pivot, year, in, the, war, on, climat, chang]","[wired, 2016, wa, a, pivotal, year, in, the, war, on, climate, change]","[wired, 2016, pivotal, year, war, climate, change]",wired 2016 pivotal year war climate change
4,1,its 2016 and a racist sexist climate change denying bigot is leading in the polls electionnight,466954,"[its, 2016, and, a, racist, sexist, climate, change, denying, bigot, is, leading, in, the, polls...","[it, 2016, and, a, racist, sexist, climat, chang, deni, bigot, is, lead, in, the, poll, election...","[it, 2016, and, a, racist, sexist, climate, change, denying, bigot, is, leading, in, the, poll, ...","[2016, racist, sexist, climate, change, denying, bigot, leading, polls, electionnight]",2016 racist sexist climate change denying bigot leading polls electionnight
